In [4]:
import pandas as pd
import pymysql

In [3]:
file_path = '../dataSet/diabetes.csv'
df = pd.read_csv(file_path)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# MariaDB 연결 설정
# 연결 함수
def connect_to_mariadb():
    return pymysql.connect(
        host='localhost',
        user='root',
        password='root',
        database='test',
        charset='utf8mb4'
    )

In [7]:
# 연결테스트
connection = connect_to_mariadb()
print('연결됨')
connection.close()

연결됨


In [9]:
# 테이블 및 컬럼 생성
def create_table_from_csv(connection,df,dtypes):
    cursor = connection.cursor()
    
    # 테이블 생성 SQL 동적 생성
    columns =[]
    for col, dtypes in zip(df.columns, df.dtypes):
        if 'int' in str(dtypes):
            col_type ='INT'
        elif 'float' in str(dtypes):
            col_type='FLOAT'
        else:
            col_type='VARCHAR(255)'
        columns.append(f"`{col}` {col_type}")
    columns_sql =",".join(columns)
    create_table_sql = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns_sql});"
    
    cursor.execute(create_table_sql)
    connection.commit()
    print(f"테이블 '{table_name}이 생성되었습니다.")
    

In [10]:
table_name ='diabetes'
connection =connect_to_mariadb()
create_table_from_csv(connection,df,table_name)
connection.close()

테이블 'diabetes이 생성되었습니다.


In [11]:
# CSV 데이터를 MariaDB로 업로드
def insert_data_to_table(connection,df,table_name):
    cursor = connection.cursor()
    
    # 데이터 삽입 SQL 동적 생성
    placeholders =",".join(["%s"] * len(df.columns))
    insert_sql = f"INSERT INTO `{table_name}` ({','.join(df.columns)}) VALUES ({placeholders})"
    
    #데이터 삽입 실행
    for _, row in df.iterrows():
        cursor.execute(insert_sql,tuple(row))
        
    connection.commit()
    print(f"테이블 '{table_name}'에 데이터가 삽입되었습니다") 

In [12]:
# 데이터 삽입 실행
connection = connect_to_mariadb()
insert_data_to_table(connection,df,table_name)
connection.close()

테이블 'diabetes'에 데이터가 삽입되었습니다


In [15]:
# MariaDB에서 데이터 읽기
def fetch_data_from_table(connection,table_name):
    query = f'SELECT * FROM `{table_name}`'
    df = pd.read_sql(query,connection)
    return df
    

In [18]:
# 데이터 읽기
connection = connect_to_mariadb()
table_name='diabetes'

# 데이터 읽기
df_from_db = fetch_data_from_table(connection,table_name)

# 읽어온 데이터 확인
print(df_from_db)
connection.close()

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

C:\Users\human-07\AppData\Local\Temp\ipykernel_14252\1463501854.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,connection)
